In [1]:
import numpy
import numpy as np
import time
import datetime
import netCDF4
import matplotlib.pylab as plt
import os,sys,glob
# sys.path.append(os.getcwd()+'/../adjust/rasotools/')
# from utils import *
from multiprocessing import Pool
#import odb
from eccodes import *
from functools import partial
from collections import OrderedDict
import subprocess
import json
import gzip
import copy
import pickle
import xarray as xr

import pandas as pd
import matplotlib
import matplotlib.pylab as plt
import matplotlib.pyplot as maplt
matplotlib.rcParams.update({'font.size': 20})

plt.rcParams['lines.linewidth'] = 3

import warnings
warnings.filterwarnings('ignore')

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua
sys.path.append(os.getcwd()+'/../')
import metutils as mu
import trajectory as trj
 

In [2]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [3]:
import trajectory as trj

In [4]:
stat = '11035'
statlist = glob.glob('/mnt/users/scratch/leo/scratch/converted_v8/*' + stat + '*_CEUAS_merged_v1.nc')
file = eua.CDMDataset(filename = statlist[0])

igra_file = glob.glob('/scratch/das/federico/COP2_HARVEST_APRIL2022/igra2/*'+stat+'*')[0]
i_file = eua.CDMDataset(filename = igra_file)


In [36]:
statlen = len(file.observations_table.observed_variable[:])
latd = np.full(statlen, np.nan)
lond = np.full(statlen, np.nan)
timed = np.full(statlen, np.nan)

failcount = 0
slat = file.observations_table.latitude[0]
slon = file.observations_table.longitude[0]
for i in range(len(file.header_table.report_id[:])-1):
    
    t_idx_s = file.recordindices['126'][i]
    t_idx_e = file.recordindices['126'][i+1]
    u_idx_s = file.recordindices['139'][i]
    u_idx_e = file.recordindices['139'][i+1]
    v_idx_s = file.recordindices['140'][i]
    v_idx_e = file.recordindices['140'][i+1]

    # -----------
    
    if (u_idx_s == u_idx_e) or (u_idx_s == u_idx_e) or (t_idx_s == t_idx_e):
        # replace with nan filling
        continue

    # -----------
        
    repid = file.header_table.report_id[i]
    u = file.observations_table.observation_value[u_idx_s:u_idx_e]
    v = file.observations_table.observation_value[v_idx_s:v_idx_e]
    t = file.observations_table.observation_value[t_idx_s:t_idx_e]
    z_coordinate_t = file.observations_table.z_coordinate[t_idx_s:t_idx_e]
    z_coordinate_u = file.observations_table.z_coordinate[u_idx_s:u_idx_e]
    z_coordinate_v = file.observations_table.z_coordinate[v_idx_s:v_idx_e]
    
    
    # find shortest array
    z_coords = [z_coordinate_t, z_coordinate_u, z_coordinate_v]
    shortest_zc = 0
    
    for k in range(len(z_coords)-1):
        if len(z_coords[k+1]) < len(z_coords[shortest_zc]):
            shortest_zc = k+1
    
#     print(shortest_zc, len(z_coords[shortest_zc]))
    # if shortest array < 7 -> skip
    if len(z_coords[shortest_zc]) < 7:
        # replace with nan filling
        continue
        
    # -----------
        
    # check if it's neccessary to interpolate
    check = (np.array(z_coordinate_t) != np.array(z_coordinate_u))
    if not (isinstance(check, bool)):
        check = check.all()
    if check:
        u_new = []
        v_new = []
        
        for k in range(len(z_coordinate_t)):
            if z_coordinate_u[0] < z_coordinate_t[k] < z_coordinate_u[-1]:
                u_new.append(np.interp(z_coordinate_t[k], z_coordinate_u, u))
                v_new.append(np.interp(z_coordinate_t[k], z_coordinate_v, v))
            else:
                u_new.append(np.nan)
                v_new.append(np.nan)
            
#         print(z_coordinate_u)
#         print(u)
#         print(v)
        u = np.array(u_new)
        v = np.array(v_new)
#         print()
#         print(z_coordinate_t)
#         print(u)
#         print(v)
#         print()
#         print()
    
    # -----------
    
    input_df = pd.DataFrame({'t':t, 'u':u, 'v':v, 'p':z_coordinate_t, 'idx':np.array(range(t_idx_s, t_idx_e))})
    # flip for ascending order
    input_df = input_df.dropna().iloc[::-1].reset_index()
#     display(input_df)
    
            
#     print(repid)

#     print(pd.to_datetime(file.observations_table.date_time[t_idx_s], unit='s', origin='1900-01-01') )
#     print(pd.to_datetime(file.observations_table.date_time[t_idx_e], unit='s', origin='1900-01-01') )
#     print(pd.to_datetime(file.observations_table.date_time[t_idx_s+1], unit='s', origin='1900-01-01') )
#     print(pd.to_datetime(file.observations_table.date_time[t_idx_e-1], unit='s', origin='1900-01-01') )
    
    date_time = file.observations_table.date_time[t_idx_s]
    dt_date = pd.to_datetime(date_time, unit='s', origin='1900-01-01')
#     print(dt_date)
    
    #check if < 2013:
    if dt_date.year < 2013:
        #if not already igra data -> select igra datetime
        if int(repid[0]) != 3:
            dups = file.header_table.duplicates[i]
            dups = dups[dups != b'']
            dups = dups[dups != b',']
            #iterate through all duplicates
            for j in range(0,int((len(dups)/11))):
                #if there is an igra duplicate:
                if int((dups[(j*11):((j+1)*11)])[0]) == 3:
                    save_id = 0
                    deci = 1
                    a = ((dups[(j*11):((j+1)*11)]))[1:]
                    for o in np.flip(a):
                        save_id += int(o)* deci
                        deci = deci*10
                    date_time = i_file.recordtimestamp[save_id]  
#                     dt_date = pd.to_datetime(date_time, unit='s', origin='1900-01-01')
                    
#                     print(i)
#                     print((dups[(j*11):((j+1)*11)]))
#                     print(save_id)
#                     print('igra_time', pd.to_datetime(i_file.recordtimestamp[save_id], unit='s', origin='1900-01-01'))
#                     print('cuon time', dt_date)
#                     print()

    
#     print('t: ', len(t), 'u: ', len(u), 'v: ', len(v), )
    
    phys_model = trj.trajectory(lat=slat, lon=slon, temperature=np.array(input_df.t), u=np.array(input_df.u), v=np.array(input_df.v), pressure=np.array(input_df.p))
#     input_df['latitude_displacement'] = phys_model[0]
#     input_df['longitude_displacement'] = phys_model[1]
#     input_df['displaced_time'] = np.array(phys_model[4]) + date_time
#     input_df['displaced_time_dt'] = pd.to_datetime(input_df['displaced_time'], unit='s', origin='1900-01-01')
#     display(input_df)
    latd[input_df.idx] = np.array(phys_model[0])
    lond[input_df.idx] = np.array(phys_model[1])
    timed[input_df.idx] = np.array(phys_model[4])




In [38]:
np.unique(latd)

array([-0.53883259, -0.49799017, -0.48860193, ...,  0.51174167,
        0.55310218,         nan])

In [12]:
a = np.array([1,2,3,5,5,6,4,4])
b = np.array([0,4,2])
c = [11,11,11]

In [13]:
a[b]=c

In [14]:
a

array([11,  2, 11,  5, 11,  6,  4,  4])

In [32]:
an_array = np.array([[1,2],[3,4]])

another_array = np.array([[1,2],[3,4], [5,6]])


an_array == another_array

False